In [1]:
## In-Class Assignment 13 - Evan Callaghan

import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## 1. Using the pandas library to read the csv data file and create a data-frame called heart
heart = pd.read_csv(file_content_stream)

heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
## 2. Removing observations with missing values

heart = heart.dropna()

In [3]:
## 3. Using male, age, currentSmoker, totChol, sysBP,BMI, heartRate, and glucose as the
## predictor variables, and TenYearCHD is the target variable to split the data into two data-frames (taking
## into account the proportion of 0s and 1s): train (80%) and test (20%).

X = heart[['male', 'age', 'currentSmoker', 'totChol', 'sysBP', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [4]:
## 4. Transforming each of the input variables in the train and test dataset to 0-1 scale

scaler = MinMaxScaler(feature_range = (0,1))
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

In [5]:
## 5. Using the train data-frame to build a LASSO model in which male, age, currentSmoker, totChol, 
## sysBP,BMI, heartRate, and glucose are the predictor variables, and TenYearCHD is the target variable

## Building the model
lasso_cv = LassoCV(cv = 5).fit(X_train, Y_train)

## Building the LASSO regression
lasso_md = Lasso(alpha = lasso_cv.alpha_).fit(X_train, Y_train)

## Extracting the coefficients
lasso_md.coef_

array([ 0.05351871,  0.26412755,  0.04369403,  0.01101356,  0.54816456,
       -0.00599441,  0.40814686])

In [7]:
## Using the top 5 variables from the LASSO model (the variables with the highest estimated coefficients in 
## absolute value) to build a logistic regression model in which TenYearCHD is the target variable

## Defining the input variables based on LassoCV coefficients
X_train_lasso = X_train[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]
X_test_lasso = X_test[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]

## Building the model
logit_md = LogisticRegression().fit(X_train_lasso, Y_train)

## Predicting on the test set
logit_preds = logit_md.predict_proba(X_test_lasso)[:, 1]

## Using 10% as cutoff value and reporting the recall
logit_preds = np.where(logit_preds < 0.1, 0, 1)

print('Recall Score of Logistic Regression Model:', recall_score(Y_test, logit_preds))

Recall Score of Logistic Regression Model: 0.9107142857142857


In [8]:
## 6. Using the train data-frame to build a random forest model (with 500 trees) in which: male,
## age, currentSmoker, totChol, sysBP,BMI, heartRate, and glucose are the predictor variables, and
## TenYearCHD is the target variable

rf_md = RandomForestClassifier(n_estimators = 500).fit(X_train, Y_train)

## Extracting feature importance values
rf_md.feature_importances_

array([0.02396792, 0.17397011, 0.02560305, 0.2023893 , 0.23388476,
       0.15054446, 0.1896404 ])

In [10]:
## Using the top 5 variables from the random forest model (the variables with the highest importance values) 
## to build a random forest model in which TenYearCHD is the target variable

## Defining the input variables based on Rnadom Forest feature importance measures
X_train_rf = X_train[['age', 'totChol', 'sysBP', 'heartRate', 'glucose']]
X_test_rf = X_test[['age', 'totChol', 'sysBP', 'heartRate', 'glucose']]

## Building the model
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train_rf, Y_train)

## Predicting on the test set
rf_preds = rf_md.predict_proba(X_test_rf)[:, 1]

## Using 10% as cutoff value and reporting the recall
rf_preds = np.where(rf_preds < 0.1, 0, 1)

print('Recall Score of Random Forest Model:', recall_score(Y_test, rf_preds))

Recall Score of Random Forest Model: 0.9285714285714286


In [ ]:
## 7. Using the results from parts (5) and (6), we would use the Random Forest Classifier model to predict 
## TenYearCHD because it has the higher recall score of the two considered models.